# Exploring demographic further

After identifying school clusters based on race/ethnicity, I'm curious to see the makeup of these schools based on other demographic buckets, e.g., language learners, students with disabilities.

In [1]:
import pandas as pd

In [2]:
df = pd.read_csv("output/clustered-2024.csv")

In [3]:
cols_to_use = [ "DBN", "Year", "% Students with Disabilities", "% English Language Learners",
              "% Poverty", "Economic Need Index" ]
demographics_df = pd.read_excel("data/demographic-snapshot-2020-21-to-2024-25-public.xlsx", sheet_name="School", usecols=cols_to_use)

In [4]:
demographics_df = demographics_df[demographics_df["Year"] == "2024-25"]
demographics_df.drop("Year", axis=1, inplace=True)

In [5]:
merged_df = df.merge(demographics_df, on="DBN", how="left")
merged_df.sample(3)

,DBN,School Name,Year,Total Enrollment,% Asian and Pacific Islander,% Black,% Hispanic,% Multi-Racial,% Native American,% White,% Missing Race/Ethnicity Data,Borough,kmeans_label,% Students with Disabilities,% English Language Learners,% Poverty,Economic Need Index
1570,75M138,P.S. 138,2024-25,709,0.028209,0.277856,0.630465,0.009873,0.004231,0.049365,0.000000,M,1,0.991537,0.282087,0.944993,Above 95%
139,02M600,The High School of Fashion Industries,2024-25,1446,0.040111,0.286999,0.593361,0.011065,0.008990,0.052559,0.006916,M,1,0.181189,0.018672,0.802213,0.787224
739,15K124,P.S. 124 Silas B. Dutcher,2024-25,307,0.035831,0.231270,0.543974,0.068404,0.003257,0.114007,0.003257,K,1,0.244300,0.195440,0.807818,0.820345


In [6]:
merged_df.to_csv("output/other-demographics-2024.csv", encoding="UTF-8", index=False)

## Students with disabilities

In [7]:
disabilities = merged_df.groupby("kmeans_label")["% Students with Disabilities"].agg(["mean", "min", "max"])
disabilities.T.style.format("{:.1%}")

kmeans_label,0,1,2,3,4
mean,18.0%,31.0%,22.5%,24.8%,22.0%
min,0.3%,0.1%,1.2%,5.9%,2.0%
max,100.0%,100.0%,52.8%,100.0%,100.0%


Note: There are 20 schools where `merged_df["% Students with Disabilities"] == 1]`. Fifteen of these are from Cluster 1.

In [8]:
merged_df[merged_df["% Students with Disabilities"] == 1][["kmeans_label", "School Name"]].sort_values(by="kmeans_label")

,kmeans_label,School Name
1578,0,P.S. Q004
1551,1,P.S. K053
1610,1,P.S. X811
1606,1,P.S. X643
1605,1,P.S. X596
1602,1,P.S. X188
1600,1,P.S. X176
1597,1,P.S. X012 Lewis and Clark School
1590,1,P.S. Q811
1588,1,John F. Kennedy Jr. School


In [9]:
merged_df.groupby("kmeans_label")["% Students with Disabilities"].std().T

kmeans_label
0    0.119135
1    0.216227
2    0.073837
3    0.135928
4    0.147033
Name: % Students with Disabilities, dtype: float64

Observations: Cluster 1 has the highest mean, and Cluster 0 has the lowest mean. Meaning, Cluster 1 may likely have a higher share of its schools population as students with disabilities.

## English language learners

In [10]:
ell = merged_df.groupby("kmeans_label")["% English Language Learners"].agg(["mean", "min", "max"])
ell.T.style.format("{:.1%}")

kmeans_label,0,1,2,3,4
mean,19.4%,16.3%,28.6%,9.6%,11.4%
min,0.0%,0.0%,0.0%,0.0%,0.0%
max,81.6%,88.0%,100.0%,36.7%,86.2%


In [11]:
merged_df.groupby("kmeans_label")["% English Language Learners"].std().T

kmeans_label
0    0.137138
1    0.114132
2    0.183837
3    0.067964
4    0.116825
Name: % English Language Learners, dtype: float64

Observations: Cluster 2 has the highest mean (more than a quarter), and Cluster 3 has the lowest. Meaning, schools under Cluster 2 has the highest share of its student population as English language learners. Cluster 3 has the lowest max percentage, with a very low std. Among the clusters, it could have the least need for English language learning programs.

## Poverty and economic need

In [12]:
merged_df["% Poverty"] = merged_df["% Poverty"].astype(str).str.replace("Above 95%", "0.951")
merged_df["% Poverty"] = pd.to_numeric(merged_df["% Poverty"], errors="coerce")

poverty = merged_df.groupby("kmeans_label")["% Poverty"].agg(["mean", "min", "max"])
poverty.T.style.format("{:.1%}")

kmeans_label,0,1,2,3,4
mean,73.4%,87.4%,89.6%,85.0%,53.0%
min,9.1%,55.7%,57.2%,30.7%,5.6%
max,95.1%,95.1%,95.1%,95.1%,94.5%


In [13]:
merged_df.groupby("kmeans_label")["% Poverty"].std().T

kmeans_label
0    0.168271
1    0.084655
2    0.072965
3    0.091757
4    0.228122
Name: % Poverty, dtype: float64

In [14]:
merged_df["Economic Need Index"] = merged_df["Economic Need Index"].astype(str).str.replace("Above 95%", "0.951")
merged_df["Economic Need Index"] = pd.to_numeric(merged_df["Economic Need Index"], errors="coerce")

economic_need = merged_df.groupby("kmeans_label")["Economic Need Index"].agg(["mean", "min", "max"])
economic_need.T.style.format("{:.1%}")

kmeans_label,0,1,2,3,4
mean,69.3%,87.8%,89.1%,83.7%,52.4%
min,13.0%,47.4%,54.5%,38.9%,6.8%
max,95.1%,95.1%,95.1%,95.1%,95.1%


In [15]:
merged_df.groupby("kmeans_label")["Economic Need Index"].std().T

kmeans_label
0    0.162325
1    0.083624
2    0.076943
3    0.098638
4    0.213386
Name: Economic Need Index, dtype: float64

Observations: Cluster 4 has the lowest mean, but it also has the widest range of poverty rates among its students. This could highlight economic disparity among schools in the cluster.

Cluster 0 also has a wide range of poverty rates, but its mean is relatively higher (almost three-quarters). So, this could mean more schools here serve impoverished students, than not.

Cluster 1 and 2 have pretty similar aggregates.